In [343]:
import json
import numpy as np
import pandas as pd

import MeCab

from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

from sklearn.manifold import TSNE

import plotly.express as px
from datetime import datetime


In [344]:
stories_json = []
with open('./output_stories_0123.json', 'r') as f:
    stories_json = stories_json + json.loads(f.read())

print(len(stories_json))
print(stories_json[1].keys())

839
dict_keys(['index', 'stories_index', 'title', 'broadcasting_date', 'abstracts_list'])


In [345]:
stories_df = pd.DataFrame(stories_json)
stories_df["abstract_joined"] = stories_df["abstracts_list"].str.join(" ")
stories_df.head()

,index,stories_index,title,broadcasting_date,abstracts_list,abstract_joined
0,0325,0,「映画ドラえもん のび太の新魔界大冒険～７人の魔法使い～」,2013年3月15日,[ 何をやってもうまくいかないのび太は、「魔法（まほう）が使えたら便利で楽しくなるにちがいな...,何をやってもうまくいかないのび太は、「魔法（まほう）が使えたら便利で楽しくなるにちがいない...
1,0326,0,「ばくはつコショウ」,2013年4月12日,[ ある朝、ねぼうしたのび太に泣きつかれたドラえもんは、『ばくはつコショウ』を取り出す。この...,ある朝、ねぼうしたのび太に泣きつかれたドラえもんは、『ばくはつコショウ』を取り出す。このコ...
2,0326,1,「何が何でもお花見を」,2013年4月12日,[ みんなから家族でお花見に行った話を聞き、うらやましくてしかたのないのび太。さっそくママに...,みんなから家族でお花見に行った話を聞き、うらやましくてしかたのないのび太。さっそくママにた...
3,0327,0,「やりクリしてハワイ旅行」,2013年4月26日,[ スネ夫からハワイ旅行をじまんされたのび太は、自分もハワイに行きたいとママにたのもうとする...,スネ夫からハワイ旅行をじまんされたのび太は、自分もハワイに行きたいとママにたのもうとするが...
4,0327,1,「しずかちゃんのはごろも」,2013年4月26日,[ 学校で「天女と羽衣（はごろも）」の劇（げき）をやることになったのび太としずか。劇の最後、...,学校で「天女と羽衣（はごろも）」の劇（げき）をやることになったのび太としずか。劇の最後、天...


In [346]:
t = MeCab.Tagger('-d /opt/homebrew/lib/mecab/dic/mecab-ipadic-neologd')
def mecab_tokenizer(text):
    parsed_lines = t.parse(text).split("\n")[:-2]
    surfaces = [l.split('\t')[0] for l in parsed_lines]
    features = [l.split('\t')[1] for l in parsed_lines]
    # 原型を取得
    bases = [f.split(',')[6] for f in features]
    # 品詞を取得
    pos = [f.split(',')[0] for f in features]

    # 各単語を原型に変換する
    token_list = [b if b != '*' else s for s, b in zip(surfaces, bases)]
    # 品詞を絞り込み
    target_pos = ["名詞"]
    token_list = [t for t, p in zip(token_list, pos) if p in target_pos]
    # アルファベットを小文字に統一
    token_list = [t.lower() for t in token_list]

    return [ text for text in filter(lambda x: not x.isascii(), token_list)]

In [347]:
# ストーリーが取得できたものに絞る
stories_df = stories_df[stories_df['abstract_joined'].str.len() > 0] 
sentences = stories_df["abstract_joined"].apply(mecab_tokenizer)
print(sentences)

0      [何, のび太, 魔法, 魔法, 便利, ちがい, もしもボックス, 現実, 世界, 魔法,...
1      [朝, 寝坊, のび太, ドラえもん, 爆発, コショウ, コショウ, 行き先, そば, 人...
2      [みんな, 家族, お花見, 話, しかた, のび太, ママ, パパ, ゴルフ, 予定, の...
3      [スネ夫, ハワイ旅行, じまん, のび太, 自分, ハワイ, ママ, 給料日, 前, お金...
4      [学校, 天女, 羽衣, 羽子, ろ, 劇, げき, こと, のび太, しずか, 劇, 最後...
                             ...                        
834    [星がきれい, 夜, のび太, スネ夫, 家, 前, 庭, 星空, パーティー, 女の子, ...
835    [おなか, 食事, のび太, ご飯, ありがたみ, ママ, のび太, まじめ, 米, パン,...
836    [子供達, 鬼ごっこ, の, のび太, 自分たち, 野郎, みんな, ジャイアン, スネ夫,...
837    [５, 回, 連続, ０, 点, のび太, 来週, テスト, ０, 点, ママ, 鬼, 顔,...
838    [冬, スポーツ, ウインタースポーツ, ランド, のび太, たち, スキー, スケート, ...
Name: abstract_joined, Length: 839, dtype: object


In [365]:
keywords_dict = {}
for keyword_list in sentences.tolist():
    for keyword in keyword_list:
        if (keyword not in keywords_dict): keywords_dict[keyword] = 1
        else: keywords_dict[keyword] = keywords_dict[keyword] + 1

keywords_df = pd.DataFrame([{
    "key": key,
    "val": val
} for key,val in keywords_dict.items()])
keywords_df = keywords_df.sort_values("val", ascending=False)


,key,val
1,のび太,3916
34,ドラえもん,2419
55,こと,1331
14,の,1241
42,ジャイアン,997
...,...,...
4697,ノビタランド,1
4699,和菓子屋,1
3169,舟,1
3626,春,1


In [366]:
pd.set_option('display.max_rows', 500)
keywords_df

,key,val
1,のび太,3916
34,ドラえもん,2419
55,こと,1331
14,の,1241
42,ジャイアン,997
...,...,...
4697,ノビタランド,1
4699,和菓子屋,1
3169,舟,1
3626,春,1


In [334]:
abstract_list = sentences.tolist()
trainings = [TaggedDocument(words = data, tags = [i]) for i,data in enumerate(abstract_list)]
m = Doc2Vec(documents= trainings, dm = 1, window=8, min_count=10, workers=4)


In [335]:
vector_list = m.dv.vectors
tsne = TSNE(n_components=2)
vector_enbedded = tsne.fit_transform(vector_list)
print(vector_enbedded)

/Users/takamirei/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:795: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/Users/takamirei/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:805: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



[[ 16.3089     20.467579 ]
 [  3.6349366  14.6214695]
 [ -2.440717   23.204382 ]
 ...
 [-12.352111   24.649843 ]
 [-22.052666   28.39189  ]
 [ -3.6598644  25.326548 ]]


In [336]:
vector_df = pd.DataFrame.from_dict({
    'vector': [v for v in vector_list],
    'vector_x': [v[0] for v in vector_enbedded],
    'vector_y': [v[1] for v in vector_enbedded]
})
stories_df_merged = stories_df.join(vector_df)
stories_df_merged['broadcasting_dt'] = stories_df_merged['broadcasting_date'].apply(lambda d: datetime.strptime(d, '%Y年%m月%d日'))
stories_df_merged['broadcasting_year'] = stories_df_merged['broadcasting_dt'].dt.year

In [337]:
fig = px.scatter(stories_df_merged, x='vector_x', y='vector_y', color='broadcasting_year', hover_data=["index", "title"])
fig.show()

In [338]:
df = px.data.tips()
fig = px.scatter(stories_df_merged, x='vector_x', y='vector_y', hover_data=["index", "title"], facet_row="broadcasting_year", width=800, height=2000)
fig.show()

In [339]:
sampled_df = stories_df_merged.sample(1)
print(sampled_df.head())
sampled_df = sampled_df.rename(columns={"vector": "sampled_vec"})[["sampled_vec"]]


    index  stories_index       title broadcasting_date  \
163  0405              2  「のび太の長い一日」         2015年6月5日   

                                        abstracts_list  \
163  [　図書館でしずかと待ち合わせをしたのび太。ところが、約束の時間を３０分過ぎてもしずかは現れ...   

                                       abstract_joined  \
163  　図書館でしずかと待ち合わせをしたのび太。ところが、約束の時間を３０分過ぎてもしずかは現れず...   

                                                vector   vector_x  vector_y  \
163  [-0.036082674, 0.024999296, -0.0051214634, 0.0...  22.346064 -1.859574   

    broadcasting_dt  broadcasting_year  
163      2015-06-05               2015  


In [340]:
stories_df_merged = stories_df_merged.assign(key=1).merge(sampled_df.assign(key=1), on='key').drop('key', axis=1)
stories_df_merged

,index,stories_index,title,broadcasting_date,abstracts_list,abstract_joined,vector,vector_x,vector_y,broadcasting_dt,broadcasting_year,sampled_vec
0,0325,0,「映画ドラえもん のび太の新魔界大冒険～７人の魔法使い～」,2013年3月15日,[ 何をやってもうまくいかないのび太は、「魔法（まほう）が使えたら便利で楽しくなるにちがいな...,何をやってもうまくいかないのび太は、「魔法（まほう）が使えたら便利で楽しくなるにちがいない...,"[-0.049815748, 0.028853253, -0.01814021, 0.017...",16.308901,20.467579,2013-03-15,2013,"[-0.036082674, 0.024999296, -0.0051214634, 0.0..."
1,0326,0,「ばくはつコショウ」,2013年4月12日,[ ある朝、ねぼうしたのび太に泣きつかれたドラえもんは、『ばくはつコショウ』を取り出す。この...,ある朝、ねぼうしたのび太に泣きつかれたドラえもんは、『ばくはつコショウ』を取り出す。このコ...,"[-0.04915992, 0.029298704, -0.017262228, 0.010...",3.634937,14.621469,2013-04-12,2013,"[-0.036082674, 0.024999296, -0.0051214634, 0.0..."
2,0326,1,「何が何でもお花見を」,2013年4月12日,[ みんなから家族でお花見に行った話を聞き、うらやましくてしかたのないのび太。さっそくママに...,みんなから家族でお花見に行った話を聞き、うらやましくてしかたのないのび太。さっそくママにた...,"[-0.055692557, 0.028093893, 0.000119145414, 0....",-2.440717,23.204382,2013-04-12,2013,"[-0.036082674, 0.024999296, -0.0051214634, 0.0..."
3,0327,0,「やりクリしてハワイ旅行」,2013年4月26日,[ スネ夫からハワイ旅行をじまんされたのび太は、自分もハワイに行きたいとママにたのもうとする...,スネ夫からハワイ旅行をじまんされたのび太は、自分もハワイに行きたいとママにたのもうとするが...,"[-0.029875167, 0.02302562, -0.0075788843, 0.00...",17.223001,-14.664090,2013-04-26,2013,"[-0.036082674, 0.024999296, -0.0051214634, 0.0..."
4,0327,1,「しずかちゃんのはごろも」,2013年4月26日,[ 学校で「天女と羽衣（はごろも）」の劇（げき）をやることになったのび太としずか。劇の最後、...,学校で「天女と羽衣（はごろも）」の劇（げき）をやることになったのび太としずか。劇の最後、天...,"[-0.029889552, 0.01129697, -0.008008632, 0.003...",7.262502,-18.343987,2013-04-26,2013,"[-0.036082674, 0.024999296, -0.0051214634, 0.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...
834,0741,1,「夜空に輝くピザ・ギョーザ」,2023年1月14日,[星がきれいにかがやく夜、のび太がスネ夫の家の前を通ると、庭で星空パーティーが開かれていた。...,星がきれいにかがやく夜、のび太がスネ夫の家の前を通ると、庭で星空パーティーが開かれていた。女...,"[-0.06561611, 0.0540737, -0.022495447, 0.00679...",-13.347775,20.424484,2023-01-14,2023,"[-0.036082674, 0.024999296, -0.0051214634, 0.0..."
835,0742,0,「ありがたみわかり機」,2023年1月21日,[おなかがいっぱいだからと食事を残すのび太に、ご飯が食べられるありがたみについて話すママ。と...,おなかがいっぱいだからと食事を残すのび太に、ご飯が食べられるありがたみについて話すママ。とこ...,"[-0.02618977, 0.016252154, -0.004147919, 0.010...",9.314423,-8.573564,2023-01-21,2023,"[-0.036082674, 0.024999296, -0.0051214634, 0.0..."
836,0742,1,「のび泥棒をタイホせよ！」,2023年1月21日,[小さな子どもたちがおにごっこをして遊んでいるのを見かけたのび太は、自分たちもやろうとみんな...,小さな子どもたちがおにごっこをして遊んでいるのを見かけたのび太は、自分たちもやろうとみんなを...,"[-0.06666636, 0.04871042, -0.01798926, 0.01867...",-12.352111,24.649843,2023-01-21,2023,"[-0.036082674, 0.024999296, -0.0051214634, 0.0..."
837,0743,0,「鬼のパンツはいいパンツ？」,2023年1月28日,[５回連続で０点を取ってしまったのび太。来週のテストも０点だったら、ママが鬼（おに）になって...,５回連続で０点を取ってしまったのび太。来週のテストも０点だったら、ママが鬼（おに）になってし...,"[-0.03994408, 0.026811903, -0.01896643, -0.001...",-22.052666,28.391890,2023-01-28,2023,"[-0.036082674, 0.024999296, -0.0051214634, 0.0..."


In [341]:
from scipy.spatial.distance import cosine

stories_df_merged["cosine_similarity"] =  stories_df_merged.apply(lambda v: 1 - cosine(v["vector"], v["sampled_vec"]), axis=1)
stories_df_merged_sort = stories_df_merged[["title", "abstract_joined", "cosine_similarity"]].sort_values("cosine_similarity", ascending=False)


In [342]:
stories_df_merged_sort

,title,abstract_joined,cosine_similarity
163,「のび太の長い一日」,図書館でしずかと待ち合わせをしたのび太。ところが、約束の時間を３０分過ぎてもしずかは現れず...,1.000000
420,「クジラとまぼろしのパイプ島」,夏休み最後の日、「どこかに出かけて、何かおもしろいことをしたい」とのび太にたのまれたドラえも...,0.977423
423,「時を走れ！タイム借り物競走」,明日の運動会でやる借り物競走の練習をするのに、何かいい方法はないかとのび太たちに相談されたド...,0.976621
304,「映画ドラえもん 新・のび太の日本誕生」,家でも学校でも叱られてばかりののび太は、家出を決意するが、どこに行っても持ち主がいることを知...,0.975451
746,「映画ドラえもん のび太の新恐竜」,恐竜（きょうりゅう）博の化石発掘（はっくつ）体験で見つけたひとつの化石を、「ぜったいに恐竜の...,0.975288
...,...,...,...
223,「のび太はうるさいナベ奉行」,\r\n ジャイアンの家で、闇鍋（やみなべ）を食べることになったのび太たち。電気を消して真っ...,-0.874010
149,「わすれろ草」,空き地にジャイアンとスネ夫の似顔絵を落書きし、みんなに見せびらかしていたのび太。するとそこ...,-0.879811
206,「脱出!!巨大クリスマスケーキ」,\r\n クリスマス会に参加するため、のび太がめずらしくおしゃれをしていると、ドラミがクリス...,-0.927835
823,「リフトストック」,部屋でステッキのようなものを手入れするドラえもんに、それは何かとたずねるのび太。ドラえもんは...,-0.959550
